# Don't use this notebook, to be implemented in the future.


In [ ]:
import sys
import os

# add to path the father directory
try:  # if py file
    sys.path.append(os.path.abspath(os.path.dirname(os.path.dirname(__file__))))
except:  # if ipynb file will run this
    sys.path.append(os.path.abspath(".."))

from typing import List
import crafter
import stable_baselines3
from stable_baselines3.common.callbacks import (
    CheckpointCallback,
    EvalCallback,
    CallbackList,
)
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import plot_results
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import sys
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

from crafter_module.crafter_gymnasium import CrafterGymnasium
from utils.llm_skills_args import Skill
from utils import util_funcs, constants
from stable_baselines3.common.vec_env import SubprocVecEnv
from nlp_module.option_policy import (
    OptionPolicy,
    OptionPolicyResponse,
    DeepseekClassifier,
)
from utils.llm_skills_args import LLM_Skills_Args

## Uncomment the option you want, with\without skills

### Primitive actions only:


In [ ]:
# primitive example:
model_path = "../results/crafter_primitive_ppo/primitive_1M/final_model.zip"
env_kwargs = {
    "raw_env_kwargs": constants.DEFAULT_RAW_ENV_KWARGS,
    "verbose": 0,
    "print_file_path": None,
}

### With skills example:


In [ ]:
#
model_path = "../results/crafter_primitive_ppo/exp0_skills/final_model.zip"
exp_dir = os.path.dirname(model_path)
logs_dir = os.path.join(exp_dir, "logs")
print_file_path = os.path.join(logs_dir, "external_eval_prints.txt")
mask_actions_indices = [0]

skills_descriptions_dir = "../skills_gen/option_policies_descriptions"
skills: List[Skill] = util_funcs.load_skills(skills_descriptions_dir)
for i, skill in enumerate(skills):
    print(f"{i}: {skill}")

verbose = 1
option_policy = DeepseekClassifier(
    model="deepseek-chat", verbose=verbose, print_file_path=print_file_path
)
num_steps_pass_llm = 0
default_action_index = 5
llm_skills_args = LLM_Skills_Args(
    skills=skills,
    option_policy=option_policy,
    num_steps_pass_llm=num_steps_pass_llm,
    default_action_index=default_action_index,
)
env_kwargs = {
    "raw_env_kwargs": constants.DEFAULT_RAW_ENV_KWARGS,
    "llm_skills_args": llm_skills_args,
    "verbose": verbose,
    "print_file_path": print_file_path,
    "mask_actions_indices": mask_actions_indices,
}

In [ ]:
env = CrafterGymnasium(
    **env_kwargs,
)

model = PPO.load(model_path, env=env)
print(f"Model trained for {model.num_timesteps} timesteps")

### Uncomment the evalution method you want to perform:


In [ ]:
# # Evaluate the agent using sb3 func that is not parallel and returns only mean and std:
# mean_reward, std_reward = evaluate_policy(
#     model, env, n_eval_episodes=10, deterministic=True
# )
# print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# Equivalent to their implementation, its not paralel!
def eval_episodes(model, env, num_episodes=5):
    rewards = []
    episodes_lengths = []
    for ep_idx in range(num_episodes):
        print(f"\n\n\n @@@@@ Evaluating episode: {ep_idx} @@@@@ \n\n")
        obs, info = env.reset()
        done = False
        episode_reward = 0
        step_idx = 0
        while not done:
            print(f"Step: {step_idx} of episode {ep_idx}")
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step_idx += 1
        rewards.append(episode_reward)
        episodes_lengths.append(step_idx)
    return rewards, episodes_lengths


rewards, episodes_lengths = eval_episodes(model, env, num_episodes=5)

In [ ]:
print(rewards)
print(f"Mean reward: {np.mean(rewards):.2f} +/- {np.std(rewards):.2f}")
print(
    f"Mean episode length: {np.mean(episodes_lengths):.2f} +/- {np.std(episodes_lengths):.2f}"
)

In [ ]:
def eval_episodes_parallel(
    model, env_ctor, num_episodes_eval_in_parallel=5, eval_env_parallel_kwargs=None
):
    """
    Evaluates the model on multiple episodes in parallel.

    :param model: Trained RL model
    :param env_ctor: Constructor for the environment (e.g., Dog, not an instance)
    :param num_episodes_eval_in_parallel: Number of evaluation episodes
    :param eval_env_parallel_kwargs: Additional kwargs for creating evaluation environments
    :return: List of episode rewards
    """
    if eval_env_parallel_kwargs is None:
        eval_env_parallel_kwargs = {}

    # Create parallel environments
    def make_env():
        return env_ctor(**eval_env_parallel_kwargs)

    envs = SubprocVecEnv([make_env for _ in range(num_episodes_eval_in_parallel)])

    obs, info = envs.reset()
    dones = np.array([False] * num_episodes_eval_in_parallel)
    episode_rewards = np.zeros(num_episodes_eval_in_parallel)

    while not np.all(dones):
        actions, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = envs.step(actions)
        episode_rewards += rewards * (~dones)  # Only add reward for ongoing episodes

    envs.close()
    return episode_rewards.tolist()


rewards = eval_episodes_parallel(
    model,
    CrafterGymnasium,
    num_episodes_eval_in_parallel=10,
    eval_env_parallel_kwargs=env_kwargs,
)